##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Cómo migrar la evaluación

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/evaluator">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     Ver en TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/migrate/evaluator.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Ejecutar en Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/migrate/evaluator.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/migrate/evaluator.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a></td>
</table>

La evaluación es una parte fundamental de los modelos de medición y evaluación comparativa.

En esta guía se muestra cómo migrar tareas evaluadoras de TensorFlow 1 a TensorFlow 2. En Tensorflow 1 esta funcionalidad se implementa mediante `tf.estimator.train_and_evaluate`, cuando la API se ejecuta de forma distribuida. En Tensorflow 2, puede utilizar el `tf.keras.utils.SidecarEvaluator` incorporado, o un bucle de evaluación personalizado en la tarea del evaluador.

Hay opciones simples de evaluación en serie tanto en TensorFlow 1 (`tf.estimator.Estimator.evaluate`) como en TensorFlow 2 (`Model.fit(..., validation_data=(...))` o `Model.evaluate`). La tarea del evaluador es preferible cuando desea que sus trabajadores no cambien entre entrenamiento y evaluación, y la evaluación integrada en `Model.fit` es preferible cuando desea que su evaluación sea distribuida.


## Preparación

In [ ]:
import tensorflow.compat.v1 as tf1
import tensorflow as tf
import numpy as np
import tempfile
import time
import os

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## TensorFlow 1: Evaluación con tf.estimator.train_and_evaluate

En TensorFlow 1, puede configurar un `tf.estimator` para evaluar el estimador utilizando `tf.estimator.train_and_evaluate`.

En este ejemplo, comience por definir el `tf.estimator.Estimator` y especifique las especificaciones de entrenamiento y evaluación:

In [ ]:
feature_columns = [tf1.feature_column.numeric_column("x", shape=[28, 28])]

classifier = tf1.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 32],
    optimizer=tf1.train.AdamOptimizer(0.001),
    n_classes=10,
    dropout=0.2
)

train_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train.astype(np.int32),
    num_epochs=10,
    batch_size=50,
    shuffle=True,
)

test_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test.astype(np.int32),
    num_epochs=10,
    shuffle=False
)

train_spec = tf1.estimator.TrainSpec(input_fn=train_input_fn, max_steps=10)
eval_spec = tf1.estimator.EvalSpec(input_fn=test_input_fn,
                                   steps=10,
                                   throttle_secs=0)

Posteriormente, entrene y evalúe el modelo. La evaluación se ejecuta de forma sincrónica entre el entrenamiento porque está limitada como una ejecución local en este cuaderno y alterna entre el entrenamiento y la evaluación. Sin embargo, si el estimador se utiliza de forma distribuida, el evaluador se ejecutará como una tarea de evaluación dedicada. Para obtener más información, consulte la [guía de migración sobre entrenamiento distribuido](https://www.tensorflow.org/guide/migrate/multi_worker_cpu_gpu_training).

In [ ]:
tf1.estimator.train_and_evaluate(estimator=classifier,
                                train_spec=train_spec,
                                eval_spec=eval_spec)

## TensorFlow 2: Evaluación de un modelo de Keras

En TensorFlow 2, si utiliza la API de Keras `Model.fit` de la API para el entrenamiento, podrá evaluar el modelo con `tf.keras.utils.SidecarEvaluator`. También puede visualizar las métricas de evaluación en TensorBoard, lo cual no se muestra en esta guía.

Para demostrarlo, empezaremos por definir y entrenar el modelo:


In [ ]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
  ])

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model = create_model()
model.compile(optimizer='adam',
              loss=loss,
              metrics=['accuracy'],
              steps_per_execution=10,
              run_eagerly=True)

log_dir = tempfile.mkdtemp()
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(log_dir, 'ckpt-{epoch}'),
    save_weights_only=True)

model.fit(x=x_train,
          y=y_train,
          epochs=1,
          callbacks=[model_checkpoint])

Después, evalúa el modelo usando `tf.keras.utils.SidecarEvaluator`. En el entrenamiento real, se recomienda utilizar un trabajo separado para llevar a cabo la evaluación para liberar recursos de los trabajadores para el entrenamiento.

In [ ]:
data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
data = data.batch(64)

tf.keras.utils.SidecarEvaluator(
    model=model,
    data=data,
    checkpoint_dir=log_dir,
    max_evaluations=1
).start()

## Siguientes pasos

- Para obtener más información sobre la evaluación sidecar considere leer el `tf.keras.utils.SidecarEvaluator` de los documentos de la API.
- Para considerar la alternancia entre entrenamiento y evaluación en Keras considere leer sobre [otros métodos incorporados](https://www.tensorflow.org/guide/keras/train_and_evaluate).